In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK


In [145]:
print_to_excel = True

In [111]:
data_BRFHRG = pd.read_excel('BRFHRG FX Exposure 29.04.2024.xlsx', sheet_name='Balance Sheet')
data_BRFTO = pd.read_excel('BRFTO FX Exposure Model 27.04.24.xlsx', sheet_name='Balance Sheet')
query = """
SELECT "amount","presentAmountUSD","orgName","companyCode",
  CASE WHEN "bscCode" in ('130','135','BSC130','BSC135') THEN 'Trade receivables'
       WHEN "bscCode" in ('160','165','BSC160','BSC165') THEN 'Advances to suppliers'
       WHEN "bscCode" in ('170','175','BSC170','BSC175') THEN 'Capital advances'
       WHEN "bscCode" in ('140','145','BSC140','BSC145') THEN 'Other receivables'
       WHEN "bscCode" in ('330','BSC330') THEN 'Provisions'
       WHEN "bscCode" in ('300','305','BSC300','BSC305') THEN 'Trade payables'
       WHEN "bscCode" in ('310','315','BSC310','BSC315') THEN 'Other payables'
       WHEN "bscCode" in ('360','365','BSC360','BSC365') THEN 'Advances received'
       WHEN "bscCode" in ('350','355','BSC350','BSC355') THEN 'Loans received'
       WHEN "bscCode" in ('210','215','BSC210','BSC215') THEN 'Loans issued'
       WHEN "bscCode" in ('340','BSC340') THEN 'Tax'
       WHEN "bscCode" in ('410','BSC410') THEN 'Pension'
       ELSE 'Unknown'
    END AS "accountPurpose"
FROM xxwh."unifiedARReport"
WHERE "reportDate"=TO_DATE('2024-04-30','YYYY-MM-DD')
AND "companyCode" in ('BRFHRG','BRFTO')
union all select
"amount","presentAmountUSD","orgName","companyCode",
CASE
        WHEN "glAccount" like '63%'
        THEN 'Provisions'
        WHEN ("glAccount" IN (5201010100, 5201010110, 5201010200, 5201011100, 5201011110, 5201011200))
        THEN 'Trade payables'
        WHEN ("glAccount" IN (5501000000, 5501010000, 5502000000, 5503020000, 5503030000, 5503030010, 5503010000))
        THEN 'Other payables'
        WHEN ("glAccount" like '62%' or "glAccount" IN (5301010000, 5301010010, 7620002000, 0035100000))
        THEN 'Advances received'
        WHEN ("glAccount" like '60%' or "glAccount" IN (5101010100, 5101020110, 5101021110, 5104040000, 5109000000, 5503050000, 5505010200, 5505010210, 5505011210, 5505020100, 
        7610101000, 7610402000, 7610404000, 7610405000, 7610601000, 7610603000, 7610691000, 7619000000, 7620002000, 0033970000, 0033500000, 0033600000, 0033960000, 0033140000,
        0033130000, 0033120000, 0033110000, 0033800000, 0033950000, 0033970100))
        THEN 'Other payables'
        WHEN ("glAccount" IN (0032200000, 0032310000))
        THEN 'Pension'
        WHEN ("glAccount" IN (0032300000, 0032150000, 0032140000, 0032130000, 0032120000, 0032110000, 0031500000, 0033970200))
        THEN 'Social'
        WHEN ("glAccount" IN (0031940000, 0031930000, 0031920000, 0031910000, 0031800000, 0031700000, 0031600000, 0031330000, 0031320000, 0031220000, 0031210000, 0031100000))
        THEN 'Tax'
        WHEN ("glAccount" IN (0030200000))
        THEN 'Loans received'
        ELSE 'Unknown'
    END AS "accountPurpose"
FROM xxwh."unifiedARReport"
WHERE "reportDate"=TO_DATE('2024-04-30','YYYY-MM-DD')
AND "companyCode" in ('BRFHRG','BRFTO')
union all select
"amount","presentAmountUSD","orgName","companyCode",
CASE
        WHEN ("glAccount" like '62%' or "glAccount" IN (7699200310, 7699100210,4601000000, 4611000000, 1501010009, 1501010019, 1501010109, 1501010119, 2201010000,
        2201010008, 2201010009, 2201010010, 2201010018, 2201010019, 2201011000, 2201011008, 2201011009, 2201011010, 2201011018, 2201011019, 2201010098, 2201010099,
        2201020000, 2201020008, 2201030000, 2201030008, 2201010090, 2201010190, 2110000000, 2110000900, 2130000000, 2130000900, 2151000000, 2151000900, 2170000000,
        2170000900, 1210000000, 1210000900, 1260000000, 2181000000, 2181000900))
        THEN 'Trade receivables'
        WHEN "glAccount" IN (6002770900, 6002090900, 6002020000, 6002020099, 6002080000, 6002090000, 6002050000, 6002020900, 6002070000, 6002070099, 6002770000,
        6002050900, 6002070900, 6099000215, 6099000224, 6099000212, 6099000217, 6099000219, 6099000619, 6099000120, 6099000123, 6099000820, 6099000430, 6099000460,
        6099000470, 6099000490, 7699100410, 6099000830, 6099000810, 6002030000, 6002040000, 6002030900, 6002040900, 6099000440, 6099000450, 6099000203, 6099000615, 
        6002140000, 6002140099, 6099000616, 6002150000, 7610001900, 7610001000, 7610001099, 2301010100, 2301010108, 2301010110, 2301010118, 2301010200, 2301010208,
        2301010210, 2301010218, 2301010300, 2301010308, 2301010400, 2301010408, 2301011100, 2301011108, 2301011110, 2301011118, 2301011200, 2301011208, 2301011210,
        2301011218, 2301011300, 2301011308, 2302010000, 2302010007, 1611000000, 1613000000, 1615000000, 7610007900, 7610007000, 7610005000, 7699100620, 7699100420)
        THEN 'Advances to suppliers'
        WHEN "glAccount" IN (6002010000, 6002010099, 6002010900, 6099000612, 6099000211, 2301020100, 2301020108, 2301020110, 2301020118, 2301020200, 2301020208,
        2301020210, 2301020218, 2301020300, 2301020308, 2301020310, 2301020318, 2301021100, 2301021108, 2301021110, 2301021118, 2301021200, 2301021208, 2301021210,
        2301021218, 2301021300, 2301021308, 2301021310, 2301021318, 2301000008, 2301001008, 2302020000, 2302020007, 1612000000, 1614000000)
        THEN 'Capital advances'
        WHEN "glAccount" IN (6001089000, 7625003000, 7610003000, 7615003000, 7699100550, 6099000710, 7699200500, 7615004000, 7629009000, 7625013000, 7625013900,
        7699100200, 7629000900, 7610406000, 7610406900, 7610405000, 7610405900, 7630707000, 7622020000, 7620801000, 7620801900, 7630705000, 7629000099, 7629000000,
        7610004000, 7618000900, 7618000000, 6001080900, 6001080000, 7622010000, 7620703000, 7620704000, 7620701000, 7620701900, 7620702000, 7622030000, 7622040000,
        7699900000, 7610500000, 7610500099, 7699100240, 7630702000, 7630703000, 7630706000, 7625006000, 7625007000, 7625001000, 7625001900, 7620200000, 7620200900,
        2601030000, 2601030008, 2601040000, 2601040008, 2601050009, 2602000000, 2602000008, 2602000009, 2602000010, 2602000018, 2602000019, 2602001000, 2602001008,
        2602001010, 2602001018, 2602010000, 2602010009, 2602060100, 2602000090, 2603000000, 2603000100, 36307060, 1284000000, 1284000900, 1630000000, 1282000000, 
        1282000900, 1251000000, 2601010009, 2601020009)
        THEN 'Other receivables'
        WHEN "glAccount" IN (2201910000, 1290000000, 2190000000, 2301910100, 2602900000)
        THEN 'Provisions'
        WHEN ("glAccount" IN (1022200000))
        THEN 'FX Spot'
        WHEN ("glAccount" IN (1110009000, 1270000000, 1271000000, 2204000000, 2604010100, 2604020100, 2604021110, 2604021200, 7621156000))
        THEN 'Loans issued'
        WHEN ("glAccount" IN (1410000000, 1430010000, 1430030000, 1430050000, 1430060000, 1430070000))
        THEN 'Tax'
        WHEN ("glAccount" IN (2602000020))
        THEN 'MTM Derivatives'        
        WHEN ("glAccount" IN (7610300000))
        THEN 'Dividends'        
        WHEN ("glAccount" IN (7611102000))
        THEN 'Capital expenses'       
        ELSE 'Unknown'
    END AS "accountPurpose"
FROM xxwh."unifiedARReport"
WHERE "reportDate"=TO_DATE('2024-04-30','YYYY-MM-DD')
AND "companyCode" in ('BRFHRG','BRFTO')
"""
data_UnifiedARReport = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:172: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [121]:
data_UnifiedARReport['index'] = data_UnifiedARReport.index
data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(index=['companyCode','accountPurpose'],values=['amount','presentAmountUSD','index'], aggfunc={'amount':sum,'presentAmountUSD':sum,'index':list}).rename(columns={'amount':'sum_amount','presentAmountUSD':'sum_presentAmountUSD','index':'list_index'}).reset_index()
data_UnifiedARReport_pivot = concat_columns(data_UnifiedARReport_pivot,['companyCode','accountPurpose'])

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\874643092.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(index=['companyCode','accountPurpose'],values=['amount','presentAmountUSD','index'], aggfunc={'amount':sum,'presentAmountUSD':sum,'index':list}).rename(columns={'amount':'sum_amount','presentAmountUSD':'sum_presentAmountUSD','index':'list_index'}).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\874643092.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(in

In [138]:
counter = 0
for excel_data in [data_BRFHRG, data_BRFTO]:
    counter += 1
    excel_data_work = excel_data[['Balance Name','Saldo Final']]
    excel_data_work['index'] = excel_data_work.index
    if counter == 1:
        excel_data_work['sourse'] = 'BRFHRG'
        excel_data_work_BRFHRG = excel_data_work
    else:
        excel_data_work['sourse'] = 'BRFTO'
        excel_data_work_BRFTO = excel_data_work
# concat data
excel_concat_data = pd.concat([excel_data_work_BRFHRG,excel_data_work_BRFTO])
excel_concat_data = excel_concat_data.dropna()
rename_dict = {'Trade payable':'Trade payables','Trade Receivables':'Trade receivables','Trade payables - credit assignment - International':'Trade payables','Trade payables - credit assignment Domestic':'Trade payables','Advances from customers':'Advances received'}
excel_concat_data['Balance Name'] = excel_concat_data['Balance Name'].replace(rename_dict)
# pivot
excel_concat_pivot_data = excel_concat_data.pivot_table(index=['sourse','Balance Name'],values=['Saldo Final','index'],aggfunc={'Saldo Final':'sum','index':list}).reset_index().rename(columns={'Saldo Final':'sum_Saldo_Final','index':'list_index'})
excel_concat_pivot_data = concat_columns(excel_concat_pivot_data,['sourse','Balance Name'])

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\928270489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_data_work['index'] = excel_data_work.index
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\928270489.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_data_work['sourse'] = 'BRFHRG'
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\928270489.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [149]:
excel_BD_merge_data = excel_concat_pivot_data.merge(data_UnifiedARReport_pivot,how='outer',left_on='concat_columns',right_on='concat_columns').drop(['concat_columns'],axis=1).sort_values('companyCode')
excel_BD_merge_data

,sourse,Balance Name,sum_Saldo_Final,list_index_x,companyCode,accountPurpose,sum_amount,list_index_y,sum_presentAmountUSD
14,BRFTO,Advances received,-4.684064e+08,"[201, 217]",BRFTO,Advances received,-3.839277e+08,"[102, 103, 104, 147, 148, 149, 150, 151, 152, ...",-7.501388e+07
29,BRFTO,Trade payables,-4.233163e+08,"[153, 154, 155, 157, 158]",BRFTO,Trade payables,-2.205747e+08,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",-9.072136e+07
30,BRFTO,Trade receivables,1.077937e+09,[40],BRFTO,Trade receivables,6.141487e+08,"[9345, 9346, 9347, 9348, 9349, 9350, 9351, 935...",2.172392e+08
31,NaN,NaN,NaN,NaN,BRFTO,Unknown,1.929255e+07,"[21382, 21383, 21384, 21385, 21386, 21387, 213...",1.030079e+08
0,BRFHRG,Advances received,-8.639768e+07,"[450, 451, 452, 453, 454, 455, 456]",NaN,NaN,NaN,NaN,NaN
1,BRFHRG,Borrowings Third Parties,-3.135645e+08,"[368, 369, 370, 371, 372, 373, 374, 375, 376, ...",NaN,NaN,NaN,NaN,NaN
2,BRFHRG,Cash and Applications,8.364422e+07,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN,NaN,NaN,NaN,NaN
3,BRFHRG,Derivatives Results and APV,0.000000e+00,"[86, 87, 428, 429]",NaN,NaN,NaN,NaN,NaN
4,BRFHRG,Equity,-7.548569e+06,"[550, 551, 552, 553, 554, 555, 556, 557, 558, ...",NaN,NaN,NaN,NaN,NaN
5,BRFHRG,Fixed Assets,5.524819e+08,"[285, 286, 287, 288, 289, 290, 291, 292, 293, ...",NaN,NaN,NaN,NaN,NaN


In [146]:
if print_to_excel == True:
    Output_file=f'{str(date.today())}_Brazil_UnifiedARReport.xlsx'
    excel_BD_merge_data.to_excel(Output_file, index=False)